In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'auto_economictimes_indiatimes_com'
c = Crawl()  # creating object
cl_data = dashboard['core_web_india']
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_rss_page = 0
publish_source = 'auto.economictimes.indiatimes.com'
country = 'India'
language = 'English'
images_path = []

rss_pages = c.download_page('https://auto.economictimes.indiatimes.com/rss')
rss_pages = c.scrap('"toggle_rss"(.*?)</tr></table></div></div><', rss_pages)

for _ in rss_pages.split('href=')[1:]:
    
    _.split('</td><td>')
    rss_url = c.scrap('"(.*?)"', _)
    rss_page = c.download_page(rss_url)
    
    if rss_page.startswith('Unable to fetch'):
        logger.info(rss_page) # writes error message with error code
        unable_to_fetch_rss_page += 1
        continue    
    
    logger.info(f'Fetching Rss url {rss_url}\n')
    
    for i in rss_page.split('<item>')[1:]:
        
        # article url
        source_link = re.search('<link>(.*?)</', i, flags=re.S).group(1)
        
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue
        time.sleep(random.randint(1,3)) 
        
        page = c.download_page(source_link)
        
        if page.startswith('Unable to fetch'):
            logger.info(page) # writes error message with error code
            unable_to_fetch_url += 1
            continue  
        
        source_headline = c.scrap('<title>(.*?)</title>', i)
        source_headline = source_headline.replace("<![CDATA[","").replace("]]>","")

         # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue

        logger.info(f'Fetching {source_link}\n')

        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('<pubDate>(.*?)</pubDate>', i)
            date_time_str = date_time_str.replace('+05:30','')
            date_time_str = re.sub('T|-|:','',date_time_str) 
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
            # +05:30
            ist_date_time = date_time_obj + timedelta(hours = 0,minutes = 0)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue
        
        if pub_date != created_on:
            break    

        journalist = c.scrap('"Person","name":"*(.*?)"', page)
        if not journalist: journalist = 'NA'
            
        
        source_content = c.scrap('captionrendered="1"\s*.alt=".*?"\s*/>(.*?)<div style="font-weight:bold" class="site-follow-article-detail">', page)
        source_content = c.strip_html(source_content)
        source_content = re.sub('&amp;','&',source_content,flags=re.S)
        if not source_content:
            source_content = c.scrap('<div class="post-text artcle-txt article-type-news">\s*(.*?)<div style="font-weight:bold" class="site-follow-article-detail">',page)
            source_content = c.strip_html(source_content)
            source_content = re.sub('&amp;','&',source_content,flags=re.S)
        
       #current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        
        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1
        

logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_page}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')      
        
       

https://auto.economictimes.indiatimes.com/rss/topstories
https://auto.economictimes.indiatimes.com/news/passenger-vehicle/cars/etauto-exclusive-expects-2021-to-be-full-of-surprises-with-strong-recovery-says-martin-schwenk-mercedes-benz-india/81754113?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/news/industry/can-india-catch-up-with-the-rest-of-asia-in-ev-production/81743197?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/rss/recentstories
https://auto.economictimes.indiatimes.com/news/auto-components/frances-car-component-producer-treves-to-shut-slovenia-factory-sta-says/81756267?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/news/industry/focus-on-curbing-mortality-not-lockdown-mahindra-tells-maharashtra/81755961?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/news/auto-components/sk-innovation-says-may-pull-its-battery-business-out-of-u-s-/81755381?utm_source=RS

https://auto.economictimes.indiatimes.com/news/auto-technology/81751824?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/news/auto-technology/81747313?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/rss/auto-finance
https://auto.economictimes.indiatimes.com/news/auto-finance/81690147?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/rss/automotive
https://auto.economictimes.indiatimes.com/news/automotive/off-highway/75966359?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/rss/automotive/off-highway
https://auto.economictimes.indiatimes.com/news/automotive/off-highway/75966359?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/rss/automotive/construction-equipment
https://auto.economictimes.indiatimes.com/news/automotive/construction-equipment/81367530?utm_source=RSS&amp;utm_medium=ETRSS
https://auto.economictimes.indiatimes.com/rss/automotive/f